# **FireForce VI – Modeling Environment**
**Author:** Ryan Posti  
**Course:** CS 188 – Fire Force VI Project  
**Instructor:** Professor Elaasar  

---

## **Abstract**

FireForce VI is envisioned as a sixth-generation autonomous firefighting system composed of many tightly coupled subsystems that must coordinate sensing, planning, and actuation under time pressure and uncertainty. Designing such a system-of-systems requires rigorous architectural models with formal semantics, traceability to requirements, and support for distributed engineering teams.

This paper presents the FireForce VI Modeling Environment, a cloud-native ontological modeling platform centered on the Ontological Modeling Language (OML) and implemented as a Visual Studio Code extension. The environment provides a language layer built with Langium for parsing, linking, and validating OML vocabularies; a diagram layer that translates a semantic diagram model into Sprotty/ELK diagrams; and tooling that integrates with the broader FireForce VI project environment and testing environment.

Using the FireForce VI ConOps as a running example, we show how stakeholders, requirements, missions, and capabilities are captured as OML vocabularies, visualized as diagrams, and queried via SPARQL. We demonstrate correct handling of per-end cardinalities, subset relationships, and scalar properties, and we discuss how these features support consistency checking and configuration management.

Our evaluation is qualitative: we assess the modeling environment against its success criteria of semantic correctness, visual clarity, and integration into a developer-centric Git/VS Code workflow. We also highlight technical challenges encountered in implementing the diagrams and lay out future work, including support for description models, richer visualization, and more comprehensive automated tests.

---

## **1. Introduction**

FireForce VI is a sixth-generation autonomous wildfire response system that must manage heterogeneous assets, operate over unreliable networks, and adapt to rapidly changing conditions. At this scale, ad-hoc diagrams and informal documentation are not sufficient; engineers need models that are precise, checkable, and kept in sync with code and requirements.

The specific problem addressed in this work is the lack of a cloud-native, collaborative modeling environment that combines formal ontological semantics with the day-to-day workflows of software and systems engineers. Traditional desktop modeling tools struggle with real-time collaboration, semantic reasoning over large models, and seamless integration with Git-based development practices.

We tackle this problem by building the FireForce VI Modeling Environment: an OML-based modeling platform that runs directly inside VS Code and treats OML artifacts as first-class source files. The environment provides language services (syntax highlighting, validation, navigation), automated diagram generation, and the ability to connect models to project-level requirements and test artifacts.

This paper is organized as follows. Section 2 reviews background and related work in model-based systems engineering and ontology-driven modeling. Section 3 presents the architecture and design of the modeling environment, including its language, diagram, and tooling layers. Section 4 walks through a demonstration scenario based on the FireForce VI ConOps and evaluates the environment against its success metrics. Section 5 concludes with lessons learned and outlines future work.

---

## **2. Background & Related Work**

### 2.1 Model-Based Systems Engineering for FireForce VI

The FireForce VI project adopts a model-based systems engineering (MBSE) approach in which architectural decisions, requirements, and verification artifacts are captured as structured models rather than only prose documents. For a system-of-systems such as FireForce VI, these models must span multiple subsystems and viewpoints, including sensing, communications, mission planning, and human-machine interfaces.:contentReference[oaicite:2]{index=2}

In this context, the modeling environment plays a critical role: it is the place where stakeholders such as model authors and system engineers define the vocabulary of the domain (concepts, relations, aspects, scalar properties) and reason about consistency across viewpoints.

### 2.2 Existing Modeling Tools and Their Limits

Conventional MBSE and UML/SysML tools provide rich diagramming environments but are typically delivered as heavyweight desktop applications with proprietary model stores. While they offer useful visualizations, they often lack:

- First-class support for ontological semantics (e.g., logical axioms and subset relationships).  
- Tight integration with Git-based workflows and VS Code.  
- Cloud-native collaboration and review across distributed teams.  
- Built-in automation for exporting model information into queries and dashboards.

For FireForce VI, these limitations show up as friction in keeping models synchronized with code, difficulty in automating consistency checks, and challenges in sharing models across geographically distributed teams.:contentReference[oaicite:3]{index=3}

### 2.3 OML and Ontology-Driven Modeling

The Ontological Modeling Language (OML) addresses many of these issues by treating system vocabularies as formally defined ontologies. OML distinguishes between vocabularies (types such as concepts, aspects, relations) and descriptions (instances and assertions), and it supports axioms such as cardinality restrictions and subset relationships. These constructs enable precise reasoning about the architecture and its constraints.

The FireForce VI Modeling Environment builds on OML to represent stakeholders, requirements, missions, and capabilities for the modeling environment itself. By grounding the architecture in OML, we can use semantic queries (e.g., SPARQL) to extract structured views of the system and drive automated analysis.

### 2.4 Our Contribution

Within this landscape, our contribution is a developer-centric OML modeling environment that:

- Embeds OML editing and validation directly into VS Code using Langium.  
- Automatically generates diagrams from the semantic model using Sprotty and ELK.  
- Treats models as code, enabling Git-based version control, pull-request review, and CI.  
- Connects the modeling artifacts to the FireForce VI project and testing environments via shared vocabularies and queries.

---

## **3. System Architecture & Design**

### 3.1 High-Level Blueprint

At a high level, the modeling environment is organized into four layers:

1. **Language Layer (Langium).** Defines the OML grammar in a `.langium` file and generates a TypeScript abstract syntax tree (AST) and language services. These services handle parsing, linking cross-references, validation, and construction of a semantic diagram model from OML vocabularies.  
2. **Diagram Model Layer.** Maps OML elements to a `DiagramModel` consisting of nodes (concepts, aspects, relation entities) and edges (relations with per-end cardinalities, subset annotations, and labels).  
3. **Diagram Rendering Layer (Sprotty + ELK).** Converts the `DiagramModel` into a Sprotty SModel, uses ELK for automatic layout (node sizing, edge routing, label placement), and renders the diagram as an interactive SVG in a VS Code webview.  
4. **Tooling & Integration Layer.** Provides VS Code commands, npm build scripts, unit tests, and a sample workspace so that engineers can generate diagrams and run validation in a reproducible way.



### 3.2 Interfaces with the FireForce VI Ecosystem

The modeling environment interfaces with other FireForce VI work packages through shared artifacts:

- **Inputs.** OML vocabularies and description documents stored in Git repositories. These include ConOps models (stakeholders, requirements, missions, capabilities) and domain vocabularies used by other subsystems.  
- **Outputs.**  
  - Diagnostic information (parse errors, validation messages).  
  - Diagrams that summarize structural relationships between concepts.  
  - Queryable model data exposed via SPARQL over the underlying RDF/OWL representation.

From the ConOps perspective, key stakeholders include:

- **Model Authors**, who create vocabularies, concepts, relations, and axioms.  
- **System Engineers**, who analyze consistency across viewpoints using diagrams and queries.  
- **Tool Administrators**, who maintain the extension, its configuration, and deployment.  
- **Distributed Engineering Teams**, who collaborate through Git, pull requests, and shared diagrams.

In [1]:
from utilities import *
import pandas as pd
pd.set_option('display.max_colwidth', None)
df = dataframe("stakeholders.json")
df

,stakeholder_id,stakeholder_desc
0,model-author,Developers working in the Fireforce project creating and editing OML models
1,system-engineer,Analyzes OML models and ensures consistency across viewpoints
2,tool-admin,A non-user stakeholder who needs to maintain the modeling environment


Their needs translate into system requirements such as:

- **R1 – Vocabulary Definition.** Ability to define concepts, aspects, relations, scalars, and axioms in OML.  
- **R2 – Analysis Assurance.** Automated detection of logical inconsistencies and broken references.  
- **R3 – Configuration & Version Control.** Support for collaborative editing, branching, and history.  
- **R4 – Performance.** Interactive latency for validation and diagram generation.

In [2]:
from utilities import *
import pandas as pd
pd.set_option('display.max_colwidth', None)
df = dataframe("requirements.json")
req_df = df[["req_id", "req_desc"]].drop_duplicates()
req_df

,req_id,req_desc
0,R1_Vocabulary_Definition,"The user must be able to define concepts, relations, and axioms."
1,R2_Analysis_Assurance,The system must verify model consistency and logical correctness.
3,R3_Version_Control,All model descriptions must be versioned and managed collaboratively.
4,R4_Peformance_Timing,Critical analysis tasks must return results in under X seconds.


These requirements, in turn, support missions **M1: Rigorous Authoring** and **M2: Automated System Assurance**, and capabilities **C1: Model Authoring**, **C2: Consistency Checking**, and **C3: Configuration Management**.

In [4]:
from utilities import *
import pandas as pd

pd.set_option('display.max_colwidth', None)

df = dataframe("capabilities.json")
cap_req = (
    df
    .groupby(["cap_id", "cap_desc"])["req_id"]
    .apply(
        lambda ids: ", ".join(
            sorted(
                {str(x) for x in ids if pd.notna(x)}
            )
        )
    )
    .reset_index(name="requirements")
)

cap_req

,cap_id,cap_desc,requirements
0,C1_Model_Authoring,"The ability for users to define and edit the formal logical constructs (concepts, relations) of domain vocabularies within the environment",R1_Vocabulary_Definition
1,C2_Consistency_Checking,"The specific functional ability to execute automated logical analysis (using Description Logic or similar techniques) to verify that models are consistent and logically correct, often subjected to performance timing constraints","R2_Analysis_Assurance, R4_Peformance_Timing"
2,C3_Configuration_Management,"The ability to handle collaborative editing, versioning, and secure management of ontological models across teams, typically involving integration with external version control systems",R3_Version_Control
3,M1_Rigorous_Authoring,"This mission focuses on providing the user with an environment capable of logically defining domain terms (concepts, relations, and axioms) to create structured and unambiguous ontological models",
4,M2_Automated_System_Assurance,"This mission ensures the quality and correctness of the models created by supporting automated verification, consistency checking, and logical analysis, often requiring specific performance timing",


The **Capabilities** specifically are also dependent on each other requiring the previous one to be completed before the next one can be implemented. 

In [ ]:
from utilities import *
import pandas as pd
import re

pd.set_option('display.max_colwidth', None)

df = dataframe("capability-dependencies.json")

all_cap_ids = set(df["cap1_id"].dropna().unique()) | set(df["cap2_id"].dropna().unique())

def make_safe_id(raw_id: str) -> str:
    return re.sub(r"[^0-9A-Za-z_]", "_", str(raw_id))

cap_id_map = {cid: make_safe_id(cid) for cid in all_cap_ids}

lines = []
lines.append("@startuml")
lines.append("skinparam dpi 150")
lines.append("top to bottom direction")
lines.append("")

for cap_id in sorted(all_cap_ids):
    alias = cap_id_map[cap_id]
    lines.append(f'class "{cap_id}" as {alias} <<Capability>>')

lines.append("")

for _, row in df.iterrows():
    cap1 = row["cap1_id"]
    cap2 = row["cap2_id"]
    if pd.notna(cap1) and pd.notna(cap2):
        alias1 = cap_id_map[cap1]
        alias2 = cap_id_map[cap2]
        lines.append(f'{alias1} --> {alias2} : requires')

lines.append("@enduml")

uml_text = "\n".join(lines)
diagram(uml_text)


![Alt text](http://www.plantuml.com/plantuml/img/VP5D3e8m48NtFSN4FObet1X2CAmvGpEK2XDAMzjXmUq5JOXHSFj-lhpC594ny3WO4Nko7WCEq7Y2z78Kx3omWzenkrcbe1MJiq8eWp72eKnbvHfjv6tapWMoxG4mmgyCMLQYnveCyPJdQ_yaIsSZHTPMJRBijEfNoBQtHpelwJkrOy1begpGOgi7RVbDs_U_YM9ZVfBakuEk4FHZd6yJnLxaLV-pu0DHQDlCptW2)

The following diagram illustrates how **Stakeholders** express **Requirements** and how **Capabilities** are derived from those same **Requirements**.

In [5]:
from utilities import *
import pandas as pd
import re

pd.set_option("display.max_colwidth", None)

# 1. Load data
df_stake = dataframe("stakeholders.json")
df_req   = dataframe("requirements.json")
df_cap   = dataframe("capabilities.json")

# 2. Minimal views
stake_min = df_stake[["stakeholder_id", "stakeholder_desc"]].drop_duplicates()
req_min   = df_req[["req_id", "req_desc", "stakeholder_desc"]].drop_duplicates()
cap_min   = df_cap[["cap_id", "cap_desc", "req_id"]].drop_duplicates()

# 3. Attach stakeholder_id to each requirement row
req_with_ids = req_min.merge(
    stake_min,
    on="stakeholder_desc",
    how="left"
)

# 4. Edge lists
stake_req_edges = req_with_ids[["stakeholder_id", "req_id"]].dropna().drop_duplicates()
req_cap_edges   = cap_min[["req_id", "cap_id"]].dropna().drop_duplicates()

# 5. Node sets (using IDs only)
stake_ids = sorted(stake_min["stakeholder_id"].dropna().unique())
req_ids   = sorted(df_req["req_id"].dropna().unique())
cap_ids   = sorted(df_cap["cap_id"].dropna().unique())

def make_safe_id(raw_id: str, prefix: str) -> str:
    core = re.sub(r"[^0-9A-Za-z_]", "_", str(raw_id))
    return f"{prefix}_{core}"

stake_id_map = {sid: make_safe_id(sid, "S") for sid in stake_ids}
req_id_map   = {rid: make_safe_id(rid, "R") for rid in req_ids}
cap_id_map   = {cid: make_safe_id(cid, "C") for cid in cap_ids}

# 6. Build PlantUML diagram text
lines = []
lines.append("@startuml")
lines.append("skinparam dpi 150")
lines.append("left to right direction")

# Node declarations: IDs as labels
for sid in stake_ids:
    alias = stake_id_map[sid]
    lines.append(f'class "{sid}" as {alias} <<Stakeholder>>')

for rid in req_ids:
    alias = req_id_map[rid]
    lines.append(f'class "{rid}" as {alias} <<Requirement>>')

for cid in cap_ids:
    alias = cap_id_map[cid]
    lines.append(f'class "{cid}" as {alias} <<Capability>>')

# Edges: Stakeholder -> Requirement (expresses)
for _, row in stake_req_edges.iterrows():
    s_alias = stake_id_map.get(row["stakeholder_id"])
    r_alias = req_id_map.get(row["req_id"])
    if s_alias and r_alias:
        lines.append(f'{s_alias} --> {r_alias} : expresses')

# Edges: Requirement -> Capability (derived by)
for _, row in req_cap_edges.iterrows():
    r_alias = req_id_map.get(row["req_id"])
    c_alias = cap_id_map.get(row["cap_id"])
    if r_alias and c_alias:
        lines.append(f'{r_alias} --> {c_alias} : derived by')

lines.append("@enduml")

uml_text = "\n".join(lines)
diagram(uml_text)

![Alt text](http://www.plantuml.com/plantuml/img/bPJ1ReCm38RlF4NKEz9ejqjLLQtOjT94frwj55o86XAMX6cy_G85YPQ4RbVR_nVR-PETDbIPkkI1lZ9HKKLBabMCH6yl0SUB8KOInVB2a8mfJ0sJ8aWvrPeiIfaX3sbj2gaMX6foX2u4jn3PR8w6Nh6GF4ErtGuotMY3POWYPm9nKDwYC4HzOYEbFJ8hcUXrRG2wW4-IH72IAJtNdAe6tl7216l7w009-D8Mb-1NROSkKPWHRWbxGNcZcOQzrhMY8iM1vSfvGIiueTBsA8YbC4hoWJ99U16ly84NgShs7FXaTWlv076aV9WuWaDtSVlkuWP83DE4HSIqecV6cMb6X6NRgXtRe4WRY0jChoECE-jZhThg2ylj-jgBW0CLDEzQ7dZ-2W_p4472SgbahIT3EdC-ph8jaIKrcC7nPjbx4rZUNCqZz-79XE5spf9hWZ-LGgrH1zCdqukT7hnNkixyWspqrdhHrAb_Qj7bpUSTFbFDBAubEEsy9lQlOD-OaNCJ-BRGgpqkVaIuslmVORhLNZ_Z-pl63aLcl_HV)

### 3.3 Design Justification

Several design decisions shape the modeling environment:

- **OML as the modeling formalism.** OML provides explicit semantics for concepts, aspects, relations, and cardinality restrictions, enabling formal reasoning about architecture rather than only drawing diagrams.  
- **VS Code Extension Architecture.** Many engineers already use VS Code and Git-based workflows. Packaging the environment as an extension allows OML models to live alongside code, benefit from pull-request review, and participate in CI pipelines.  
- **Langium for Language Services.** Langium generates a robust language server from the OML grammar, reducing the boilerplate required to implement parsing, validation, and cross-referencing while remaining fully programmable in TypeScript.  
- **Sprotty + ELK for Diagrams.** Sprotty cleanly separates the conceptual diagram model from rendering, while ELK provides high-quality automatic layout. This combination lets the tool focus on computing the correct graph structure while delegating node/edge placement to a mature layout engine.


### 3.4 Technical Challenges

Implementing the diagram layer surfaced several technical challenges:

1. **Per-End Cardinalities.** Relation cardinalities are asymmetric; the upper and lower bounds can differ at each end. Early versions of the diagrams incorrectly treated cardinalities as symmetric, which led to misleading visuals. The current design computes and renders per-end cardinalities based on `PropertyCardinalityRestrictionAxiom` instances.  
2. **Explicit vs. Functional Defaults.** OML assumes a default functional cardinality of `[0..1]` unless overridden. The diagram computation must merge explicit restrictions with defaults without overwriting user-specified constraints.  
3. **Label Clutter.** Adding names, cardinalities, subset annotations, and scalar properties can quickly overwhelm the diagram. We mitigated this by stacking labels consistently, tuning size hints, and selectively omitting low-value details.  
4. **Sprotty/ELK Constraints.** ELK limits some combinations of label positions and edge routing styles. Achieving readable diagrams required experimenting with layout direction, spacing parameters, and node size calculations.

---

## **4. Demonstration & Evaluation**

### 4.1 Demonstration Scenario

To demonstrate the modeling environment in action, we use the FireForce VI Modeling Environment ConOps as a worked example:

1. **Authoring the ConOps Vocabulary.** A model author defines OML vocabularies for stakeholders, requirements, missions, and capabilities associated with the modeling environment.  
2. **Viewing Diagnostics and Diagrams.** The OML files are opened in VS Code with the extension enabled. The language server highlights syntax or linking errors, and the “Open Diagram” command renders an up-to-date diagram showing how capabilities (C1–C3) are derived from requirements (R1–R4) and stakeholder needs.  
3. **Exploring the Model.** The system engineer pans and zooms the diagram to inspect relation ends, per-end cardinalities, and subset relationships, verifying that they match the intended semantics of the ConOps.  
4. **Querying the Model.** A SPARQL query (Listing 1) retrieves all mission instances and their descriptions, producing a table that can be used in reports or dashboards.

This end-to-end flow illustrates how OML artifacts move from text to diagrams to structured queries within a single toolchain.


### 4.2 Example SPARQL Query

The following SPARQL query lists all mission instances and their descriptions from the FireForce VI “sierra” base ontology:

```sparql
PREFIX base: <https://fireforce6.github.io/sierra/base#>
PREFIX capability: <https://fireforce6.github.io/sierra/capability#>

SELECT DISTINCT ?mission_id ?mission_desc
WHERE {
    ?mission a capability:Mission ;
             base:description ?mission_desc .
    
    # Extract the instance name after the last # or /
    BIND(REPLACE(STR(?mission), "^.*[/#]([^/#]+)$", "$1") AS ?mission_id)
}
ORDER BY ?mission_id

```
Running this query against the ConOps model yields a concise table of missions (e.g., M1_Rigorous_Authoring, M2_Automated_System_Assurance) and human-readable descriptions, which can be cross-checked against the ConOps document and used to drive dashboards or test cases.

### 4.3 Evaluation & Results

We evaluate the modeling environment qualitatively against the success metrics implied by the ConOps:

Semantic Correctness. For the sample “family” vocabulary and the modeling environment ConOps model, diagrams correctly reflect per-end cardinalities, functional defaults, and subset relationships as specified in the OML source. Manual inspection confirmed that relation ends and scalar properties match the textual definitions.

Visual Clarity. After tuning ELK layout parameters and label stacking, diagrams remain readable even when multiple relations share a common concept. Cardinality labels no longer overlap node boundaries, and subset annotations are legible without dominating the view.

Workflow Integration. Because the environment is packaged as a VS Code extension, OML models can be stored in Git, reviewed via pull requests, and validated in CI. This matches the existing workflows of FireForce VI developers and lowers adoption friction.

The current evaluation is limited to small- to medium-sized vocabularies and qualitative assessments. Scaling to very large models and performing systematic performance measurements (e.g., diagram generation time as a function of node/edge count) remain important areas for future work.

---

# 5. Conclusion & Future Work

This paper presented the FireForce VI Modeling Environment, a VS Code-based ontological modeling platform that supports rigorous authoring and automated assurance for the FireForce VI system-of-systems. By combining OML, Langium, and Sprotty/ELK, the environment provides formal semantics, automatic diagram generation, and integration into developer-centric workflows.

From the FireForce VI ConOps perspective, the environment satisfies key requirements for vocabulary definition, analysis assurance, configuration management, and interactive performance. The demonstration scenario showed how ConOps elements such as stakeholders, missions, and capabilities can be modeled, visualized, and queried within a single toolchain.

Through this work we gained deeper understanding of OML’s vocabulary structures, the mapping from ASTs to semantic diagram models, and the practical trade-offs involved in automatic layout and label management. Perhaps most importantly, the project reinforced the value of iterative refinement: making small changes, running the build, inspecting the diagrams, and repeating until the visualization communicates the intended semantics clearly.

Future work will focus on extending the modeling environment in several directions:

Adding full support for description documents (individuals and assertions) in diagrams.

Enhancing scalar property visualization with better grouping or color-coding.

Improving styling for aspects and key relations to make them visually distinct.

Introducing interactive features such as diagram filtering, focus mode, and search.

Strengthening automated tests around diagram invariants and layout regressions.

Exposing layout configuration so users can tune diagram density and aesthetics.

Together, these enhancements will further increase the usability, expressiveness, and scalability of the FireForce VI Modeling Environment and deepen its integration with the broader FireForce VI engineering ecosystem.

---

# References
- FireForce VI Development Work Package: Modeling Environment.

- FireForce VI Project Presentation Slides.

- Ontological Modeling Language (OML) specification.

- Langium documentation.

- Sprotty documentation.

- ELK layout engine documentation.